# Proyecto (análisis dataset)

In [21]:
# --- General ---
import os
from datetime import datetime

# --- Data ---
import pandas as pd
from pandasql import sqldf
import numpy as np

# --- Conexión ---
import elasticsearch

# --- Procesamiento lenguaje: spacy ---
import spacy
from spacy.matcher import Matcher
# from spacy.matcher import PhraseMatcher

# --- Procesamiento lenguaje: gensim ---
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# --- Visualización ---
import pyLDAvis
import pyLDAvis.gensim_models  # don't skip this
# import matplotlib.pyplot as plt
import geopandas as gpd
%matplotlib inline

# ==============================================================================

# --- Configuración ---
# User password
password = os.environ.get('SOPHIA2')

# geoData
comunas = ['Ancud', 'Calbuco', 'Castro', 'Chaitén', 'Chonchi', 'Cochamó', 'Curaco de Vélez', 'Dalcahue', 'Fresia', 'Frutillar', 'Futaleufú', 'Hualaihué', 'Llanquihue', 'Los Muermos', 'Maullín', 'Osorno', 'Palena', 'Puerto Montt', 'Puerto Octay', 'Puerto Varas', 'Puqueldón', 'Purranque', 'Puyehue', 'Queilén', 'Quellón', 'Quemchi', 'Quinchao', 'Río Negro', 'San Juan de la Costa', 'San Pablo']

# --- Funciones ---
# Cargar paquete de español mediano
nlp = spacy.load("es_core_news_md")

## Leemos la información de los archivos

In [22]:
# --- Cargar Archivo ---
nombre_archivo = "data_Region_X"
from_="2022-01-01"
to_="2022-01-07"

In [23]:
archivo = nombre_archivo+"_"+from_+"_"+to_+'.csv'
df = pd.read_csv('./data/'+archivo)
df.drop('Unnamed: 0', axis=1, inplace=True)

df.head(2)

,id_news,country,media_outlet,url,title,text,date,San_Pablo,San_Juan_de_la_Costa,Río_Negro,...,Frutillar,Fresia,Dalcahue,Curaco_de_Vélez,Cochamó,Chonchi,Chaitén,Castro,Calbuco,Ancud
0,21908558.0,chile,radiosago,https://www.radiosago.cl/140-contagios-de-covi...,140 contagios de covid-19 se reportaron este j...,Así lo confirmó la autoridad sanitaria detalla...,2022-01-06,1,0,1,...,1,1,1,0,0,1,0,1,1,1
1,21908579.0,chile,radiosago,https://www.radiosago.cl/por-tormentas-electri...,Por tormentas eléctricas declaran alerta en cu...,Una Alerta Temprana Preventiva por tormentas e...,2022-01-05,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Definimos los patrones de búsqueda

In [24]:
# --- Texto de las noticias a lista ---
noticias = df.text.values.tolist()

nlp = spacy.load("es_core_news_sm")
matcher = Matcher(nlp.vocab)

# --- Definir patrones ---
# pattern: NOUN-de-NOUN
pattern_1 = [{"POS": "NOUN"},{"LOWER": "de"}, {"POS": "NOUN"}]
matcher.add("NOUN-de-NOUN", [pattern_1])

# pattern: NOUN-ADJ
pattern_2 = [{"POS": "NOUN"}, {"POS": "ADJ"}]
matcher.add("NOUN-ADJ", [pattern_2])

## Creamos filtro palabras relevantes

In [25]:
def text_to_list(noticia):
    list_of_words = []
    
    try:
        doc = nlp(noticia)

        for token in doc:
            if (token.pos_=="NOUN"):
                list_of_words.append(token.text)

        for ent in doc.ents:
            if (ent.label_ == "PER" and " " in ent.text):
                list_of_words.append(ent.text)

        matches = matcher(doc)

        for match_id, start, end in matches:
            span = doc[start:end]  # The matched span
            list_of_words.append(span.text)
    
    except Exception as e: 
        print(noticia)
        print(e)
    
    return list_of_words

## Procesamos las noticias

In [26]:
# --- Procesar texto ---
noticias_procesadas = []
for index, noticia in enumerate(noticias):
    noticia_procesada = text_to_list(noticia)
    noticias_procesadas.append(noticia_procesada)

## Prepararamos los datos de entrada de LDA
Los datos de entrada de LDA son: 
- un diccionario (variable 'id2word')
- nuestro dataset preprocesado ('noticias_procesadas')

In [27]:
# --- Crear diccionario de palabras ---
id2word = corpora.Dictionary(noticias_procesadas)
texts = noticias_procesadas

dataset_id = [id2word.doc2bow(noticia_procesada) for noticia_procesada in noticias_procesadas]

In [28]:
# --- Crear modelo LDA ---
lda_model = gensim.models.ldamodel.LdaModel(corpus=dataset_id,
                                           id2word=id2word,
                                           num_topics=5, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [29]:

# --- Generar el mejor modelo LDA ---
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        print(num_topics)
        
        model = gensim.models.ldamodel.LdaModel(corpus=dataset_id,
                                           id2word=id2word,
                                           num_topics=num_topics, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)
        
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [30]:
start=2
limit=10
step=2

In [31]:
# --- Optimal model ---
model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=dataset_id, texts=noticias_procesadas, start=start, limit=limit, step=step)

2
4
6
8


In [32]:
optimal_model = model_list[3]

## Visualizar tópicos

In [33]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(optimal_model, dataset_id, id2word)
vis

c:\Users\darko\Miniconda3\envs\myenv\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7     -0.242983 -0.015715       1        1  30.417315
4     -0.033763  0.041231       2        1  15.197288
3      0.018986  0.135790       3        1  13.364933
6     -0.037315 -0.064522       4        1  10.509491
1      0.056190 -0.041864       5        1   8.716321
5      0.029918 -0.158413       6        1   7.656443
2      0.165647 -0.039269       7        1   7.380679
0      0.043320  0.142761       8        1   6.757531, topic_info=              Term        Freq       Total Category  logprob  loglift
1168  trabajadores   37.000000   37.000000  Default  30.0000  30.0000
11             día   71.000000   71.000000  Default  29.0000  29.0000
252      comunidad   96.000000   96.000000  Default  28.0000  28.0000
106            año  151.000000  151.000000  Default  27.0000  27.0000
346       proyecto  110.000000  110.000000  Default  26.0000  26.0000
...            ...         ...         ...      ...      ...      ...
1830         etapa    6.191899   24.624449   Topic8  -5.5578   1.3140
467      situación    6.652246   37.719675   Topic8  -5.4861   0.9593
1042       entrega    6.160499   26.276055   Topic8  -5.5629   1.2440
164          parte    7.037467  103.881129   Topic8  -5.4298   0.0025
52           enero    6.513304   82.801079   Topic8  -5.5072   0.1519

[467 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
7106      6  0.795280         '
93        1  0.303564      2022
93        2  0.086733      2022
93        4  0.216832      2022
93        5  0.346930      2022
...     ...       ...       ...
33        6  0.089510  víctimas
33        7  0.447549  víctimas
2703      3  0.312182     zonas
2703      4  0.624365     zonas
4110      2  0.870756     éxito

[746 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 5, 4, 7, 2, 6, 3, 1])

## Temas principales

In [34]:
def format_topics_documents(ldamodel=None, corpus=dataset_id, texts=texts):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row_list in enumerate(ldamodel[corpus]):
        row = row_list[0] if ldamodel.per_word_topics else row_list            
        # print(row)
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['dominant_Topic', 'perc_Contribution', 'topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_documents(ldamodel=optimal_model, corpus=dataset_id, texts=noticias)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['document_No', 'dominant_Topic', 'topic_Perc_Contrib', 'keywords', 'text']
df_dominant_topic.head(5)

C:\Users\darko\AppData\Local\Temp\ipykernel_16064\2884501078.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)


,document_No,dominant_Topic,topic_Perc_Contrib,keywords,text
0,0,5,0.9766,"trabajadores, salud, empresas, día, año, perso...",Así lo confirmó la autoridad sanitaria detalla...
1,1,6,0.9164,"mascarilla, comunidad, días, respecto, trabajo...",Una Alerta Temprana Preventiva por tormentas e...
2,2,7,0.9956,"proyecto, comunidad, comuna, años, personas, a...","Patricio Vallespin, gobernador regional de Los..."
3,3,7,0.4604,"proyecto, comunidad, comuna, años, personas, a...",Un incendio se desató en la madrugada de este ...
4,4,5,0.7529,"trabajadores, salud, empresas, día, año, perso...",Un hombre falleció luego que el vehículo que c...


In [35]:
# add comuna to dataframe df_dominant_topic using merge
df_dominant_topic = pd.merge(df_dominant_topic, df)

## Revisar temas principales

In [36]:
df_dominant_topic.head(2)

,document_No,dominant_Topic,topic_Perc_Contrib,keywords,text,id_news,country,media_outlet,url,title,...,Frutillar,Fresia,Dalcahue,Curaco_de_Vélez,Cochamó,Chonchi,Chaitén,Castro,Calbuco,Ancud
0,0,5,0.9766,"trabajadores, salud, empresas, día, año, perso...",Así lo confirmó la autoridad sanitaria detalla...,21908558.0,chile,radiosago,https://www.radiosago.cl/140-contagios-de-covi...,140 contagios de covid-19 se reportaron este j...,...,1,1,1,0,0,1,0,1,1,1
1,1,6,0.9164,"mascarilla, comunidad, días, respecto, trabajo...",Una Alerta Temprana Preventiva por tormentas e...,21908579.0,chile,radiosago,https://www.radiosago.cl/por-tormentas-electri...,Por tormentas eléctricas declaran alerta en cu...,...,0,0,0,0,0,0,0,0,0,0


## Guardamos nuevamente el dataset pero ahora procesado

In [37]:
df_dominant_topic.to_csv('./data/'+nombre_archivo+"_"+from_+"_"+to_+'_dominant_topic.csv', index=False)